## Superior K-NN Classifier

See the **Cosine-Similarity Model Analysis** notebook in this repo for the context of the development of this notebook.

In this notebook, we implement a k-nearest-neighbors classifier that classifies images using a tree-based approach.  In using this approach, we notice the significant performance speedups this algorithm gives us over the standard "brute-force" algorithm, as well as performance speed-ups over the optimized Scikit-Learn K-NN classifier.

In this algorithm we implement below, everything will be the same as the previous algorithm (including the use of cosine similarity), except for the use of a tree based system in the classification step.  Recall that under the cosine similarity distance metric a value closer to one means the two vectors are "more similar" than values closer to zero.

We start by importing our libraries, and developing the updated algorithm below.

In [1]:
import numpy as np
import heapq
import collections
from collections import Counter
from bisect import bisect_left

from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, model_selection

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix  

Import MNIST Data

In [2]:
train_data = np.loadtxt("mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt("mnist_test.csv", 
                       delimiter=",")

In [3]:
train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])

### Developing the algorithm.
We start by developing a function that will return our distance tree we can use in point classification.

In [13]:
# start with producing a method, produce_tree, that will take the input data and develop a "distance tree" that we can use in classifiction
def produce_tree(data):
    """Take a data set, data, and return a distance tree array to be used for classification purposes"""
    base_len = data.shape[1]
    base = np.array([np.ones(base_len)])
    
    # distance list to sort and operate on
    dist_ref = []
    # dictionary of (distance-to-base, index value) pairs
    distances = {}

    for i in range(0, data.shape[0]):

        curr = data[i]
        dist = cosine_similarity([curr], base)
        
        # assign each to a dictionary
        distances[dist[0][0]] = i
        dist_ref.append(dist[0][0])

    odistances = collections.OrderedDict(sorted(distances.items()))
    dist_ref.sort()
    
    return dist_ref, odistances

Define our Binary search method for the classification step

In [6]:
def binary_search(a, x, lo=0, hi=None):  # can't use a to specify default for hi
    hi = hi if hi is not None else len(a)  # hi defaults to len(a)   
    pos = bisect_left(a, x, lo, hi)  # find insertion position
    return (pos if pos != hi and a[pos] == x else pos)  # don't walk off the end

Now that we have our distance tree function, we may write the core K-NN algorithm that classifies each point efficiently using a binary search method.

In [14]:
def tree_knn(k, dist_tree, dist_dict, test_data, stored_target):
    """k: number of neighbors to use for voting
    dist_tree: the array of distance values used for classification
    test_data: an unobserved image to classify
    test_target: the label for the test_data (for calculating accuracy)
    stored_data: the images already observed and available to the model
    stored_target: labels for stored_data
    """
    
    # start by taking distance of input data and base vector to get distance
    base = np.ones(len(test_data)).reshape(1, -1)
    test_data = test_data.reshape(1, -1)
    
    # this is the distance value we will use in classifying it
    similarity = cosine_similarity(base, test_data)
    
    # use binary search to find the index in dist_tree that it belongs to
    indx = binary_search(dist_tree, similarity)
    
    # now get the set of k most similar data points
    
    # when going out around indx, make sure have room to expand out without going outside of list index range
    top_room = len(dist_tree) - indx
    bot_room = indx
    k_rounded = 0
    lower_bound = 0
    upper_bound = 0
    
    if((k%2) == 0):
        k_rounded = k/2
    else:
        k_rounded = round(k/2) + 1
        
    if((bot_room >= k_rounded) and (top_room >= k_rounded)):
        lower_bound = int(indx-(k_rounded))
        upper_bound = int(indx+(k_rounded))
    
    if(bot_room < k_rounded):
        diff = k_rounded - bot_room
        lower_bound = int(indx-(k_rounded - diff))
        upper_bound = int(indx+(k_rounded + diff))
        
    if(top_room < k_rounded):
        diff = k_rounded - top_room
        lower_bound = int(indx-(k_rounded + diff))
        upper_bound = int(indx+(k_rounded - diff))
        
    # now that we know we can safely get the right sub-sequence, get the subsequence of most similar points
    most_sim = dist_tree[lower_bound:upper_bound+1]
    
    # now find the indices in our data set that correspond with the most similar points,
    # and use those corresponding data point labels to determine the classification of the 
    # new point through voting
    
    # get the indices of the most similar
    indices = []
    for i in range(0, len(most_sim)):
        curr = most_sim[i]
        val = dist_dict[curr]
        indices.append(val)
        
    # now get the corresponding labels of each of the most similar points given their indices
    labels = [stored_target[i] for i in indices]   
    labels = [i[0] for i in labels]
    
    # vote, and return prediction for every image in test_data
    pred = np.bincount(labels)
    pred = np.argmax(pred)
    
    # print table giving classifier accuracy using test_target
    return pred

Now define a function we can use to test the accuracy of our thing

In [15]:
import time

def classify(indx):
    """classifies the object in test_data[indx] by using the tree_knn method"""
    
    start = time.time()
    dist_tree, dist_dict = produce_tree(train_data)
    end = time.time()
    print("Tree Construction Time ", end-start)
    
    start = time.time()
    pred = tree_knn(8, dist_tree, dist_dict, test_data[indx], train_labels)
    end = time.time()
    print("\nPoint Classification Time ", end-start)
    
    actual = test_labels[indx]
    result = pred == actual
    
    print("\n Actual: ", actual, " Pred: ", pred)
    return result

In [16]:
classify(6)

Tree Construction Time  8.014662981033325

Point Classification Time  0.0006976127624511719

 Actual:  [4.]  Pred:  4


array([ True])

In [17]:
correct = 0
dist_tree, dist_dict = produce_tree(train_data)

for i in range(0, test_data.shape[0]):
    actual = test_labels[i]
    pred = tree_knn(4, dist_tree, dist_dict, test_data[i], train_labels)
    if(pred == actual):
        correct += 1
        
print(correct, test_data.shape[0])

2018 10000


Upon analyzing the algorithm, it is clear that this algorithm severely lacks in classification accuracy.  This is because the comparison through a base vector does not capture enough information to make it work very well.  With that said, it does achieve the goal of classifying points much faster.